# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load cities API file 
cities_df = pd.read_csv("cities2.csv")
cities_df

,Unnamed: 0,City,Lat,Long,maxtemp,humidity,cloudiness,windspeed,country,date
0,0,Ürümqi,43.8010,87.6005,-6.65,84,75,2.24,CN,1643504407
1,1,Poum,-20.2333,164.0167,81.84,76,99,9.04,NC,1643504438
2,2,Bathsheba,13.2167,-59.5167,77.61,69,40,16.11,BB,1643503853
3,3,San Cristobal,7.7669,-72.2250,65.93,84,69,3.06,VE,1643504439
4,4,Yumen,40.2833,97.2000,11.37,66,0,7.63,CN,1643504087
...,...,...,...,...,...,...,...,...,...,...
552,552,Shambu,9.5667,37.1000,51.89,55,100,1.28,ET,1643504608
553,553,Coyhaique,-45.5752,-72.0662,50.61,94,100,9.33,CL,1643504420
554,554,Darnah,32.7670,22.6367,52.45,83,100,35.68,LY,1643504609
555,555,Tegul'det,57.3064,88.1667,-3.73,100,85,2.64,RU,1643504609


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"
locations = cities_df[["Lat", "Long", "City", "humidity", "maxtemp"]]
# configure heat map with Lat long 


In [6]:

fig = gmaps.figure()

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Long"]]

# Fill NaN values and convert to float
Humidity = cities_df["humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
low_humidity_df= cities_df.loc[cities_df["humidity"] <= 15]
low_humidity_df

,Unnamed: 0,City,Lat,Long,maxtemp,humidity,cloudiness,windspeed,country,date
86,86,Saint George,37.1041,-113.5841,57.67,15,0,4.61,US,1643504022
214,214,Adrar,20.5022,-10.0711,73.89,12,100,9.37,MR,1643504069
259,259,Filingué,14.3521,3.3168,67.50,13,3,12.57,NE,1643504215
293,293,Biltine,14.5333,20.9167,61.23,15,1,10.65,TD,1643504068
330,330,Tomaszów Mazowiecki,51.5313,20.0086,43.02,10,79,29.84,PL,1643504531
345,345,Duku,11.2361,4.9073,72.88,12,86,10.29,NG,1643504537
364,364,Tombouctou,20.0000,-3.0000,71.08,11,100,13.24,ML,1643504543
426,426,Bafoulabé,13.8065,-10.8321,82.27,13,1,7.31,ML,1643504565


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df= pd.DataFrame(low_humidity_df)
hotel_df["Hotel_Name"] = ""
hotel_df.head(10)



,Unnamed: 0,City,Lat,Long,maxtemp,humidity,cloudiness,windspeed,country,date,Hotel_Name
86,86,Saint George,37.1041,-113.5841,57.67,15,0,4.61,US,1643504022,
214,214,Adrar,20.5022,-10.0711,73.89,12,100,9.37,MR,1643504069,
259,259,Filingué,14.3521,3.3168,67.50,13,3,12.57,NE,1643504215,
293,293,Biltine,14.5333,20.9167,61.23,15,1,10.65,TD,1643504068,
330,330,Tomaszów Mazowiecki,51.5313,20.0086,43.02,10,79,29.84,PL,1643504531,
345,345,Duku,11.2361,4.9073,72.88,12,86,10.29,NG,1643504537,
364,364,Tombouctou,20.0000,-3.0000,71.08,11,100,13.24,ML,1643504543,
426,426,Bafoulabé,13.8065,-10.8321,82.27,13,1,7.31,ML,1643504565,


In [14]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key":g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
       hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [15]:
hotel_df

,Unnamed: 0,City,Lat,Long,maxtemp,humidity,cloudiness,windspeed,country,date,Hotel_Name
86,86,Saint George,37.1041,-113.5841,57.67,15,0,4.61,US,1643504022,"Casablanca Resort, Casino, Golf and Spa"
214,214,Adrar,20.5022,-10.0711,73.89,12,100,9.37,MR,1643504069,
259,259,Filingué,14.3521,3.3168,67.50,13,3,12.57,NE,1643504215,
293,293,Biltine,14.5333,20.9167,61.23,15,1,10.65,TD,1643504068,Hôtel Trois Étoiles de Biltine
330,330,Tomaszów Mazowiecki,51.5313,20.0086,43.02,10,79,29.84,PL,1643504531,Vienna House Andel`s Łódź
345,345,Duku,11.2361,4.9073,72.88,12,86,10.29,NG,1643504537,Lelna Suites And Hotels
364,364,Tombouctou,20.0000,-3.0000,71.08,11,100,13.24,ML,1643504543,
426,426,Bafoulabé,13.8065,-10.8321,82.27,13,1,7.31,ML,1643504565,RESTAURANT TOUNKAGOUNA


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]
humidity=hotel_df["humidity"].astype(float)
#rating = airport_df["Airport Rating"].astype(float)

In [19]:
# Store latitude and longitude in locations
markers =gmaps.marker_layer(locations, info_box_content=hotel_info)

# Fill NaN values and convert to float



# Add marker layer ontop of heat map
# Plot Heatmap
#fig = gmaps.figure()

# Create heat layer
#heat_layer = gmaps.heatmap_layer(locations,  weights=humidity,
                                 #dissipating=False, max_intensity=300,
                                 #point_radius=1)


# Add layer
fig.add_layer(markers)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))